In [ ]:
import numpy as np
import cv2
from platform import python_version
import PIL.Image
import io
from IPython.display import clear_output, Image, display, HTML
from time import sleep
import shutil
import os

print(python_version())
print (cv2.__version__)

In [ ]:
class BackgroundSubtractor:
    def __init__(self):
        self.knn = cv2.createBackgroundSubtractorKNN()
        self.mog = cv2.bgsegm.createBackgroundSubtractorMOG() # 1st good
        self.mog2 = cv2.createBackgroundSubtractorMOG2()
        self.gsoc = cv2.bgsegm.createBackgroundSubtractorGSOC() # 2nd good
        self.gmg = cv2.bgsegm.createBackgroundSubtractorGMG() # bad
        self.kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        
    def runKNN(self, frame):
        return self.knn.apply(frame)

    def runMOG(self, frame):
        return self.mog.apply(frame)
    
    def runMOG2(self, frame):
        return self.mog2.apply(frame)

    def runGSOC(self, frame):
        return self.gsoc.apply(frame)

    def runGMG(self, frame):
        mask = self.gmg.apply(frame)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel)
        return mask

    def run(self, frame):
        return self.runGSOC(frame)
    
def showarray(a, fmt='jpeg'):
    a = np.uint8(a)
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))


backSub = BackgroundSubtractor()
# capture = cv2.VideoCapture(0)
# capture = cv2.VideoCapture('data/cps-test-videos/192.168.1.100_2020-01-28_22-24-06.mp4')
capture = cv2.VideoCapture('data/cps-test-videos/192.168.1.103_2020-01-28_22-24-07.mp4')
out = cv2.VideoWriter('data/background_substraction/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (1280,720))

if not capture.isOpened:
    print('Unable to open: ' + args.input)
    exit(0)
    
while True:
    ret, frame_img = capture.read()
    if frame_img is None:
        break
    
    frame_img = cv2.cvtColor(frame_img, cv2.COLOR_BGR2RGB)
    background_mask = backSub.run(frame_img)
#     showarray(background_mask)
    background_removed_img = cv2.bitwise_and(frame_img, frame_img, mask=background_mask)
    showarray(background_removed_img)
    background_removed_img = cv2.cvtColor(background_removed_img, cv2.COLOR_RGB2BGR)
    out.write(background_removed_img)
    clear_output(wait=True)

out.release()